# TD4 – Exercice 3 : Équation de diffusion-réaction 1D
Ce notebook contient :
- Rappel de la solution analytique
- Implémentation du schéma centré
- Résolution pour N quelconque
- Analyse de l'erreur
- Impact de la division de h

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Solution analytique
def solution_exacte(x, a):
    return np.sinh(np.sqrt(a) * x) / np.sinh(np.sqrt(a))

## Schéma centré et résolution numérique

In [ ]:
def solve_diffusion_reaction(a, N):
    h = 1 / (N + 1)
    x = np.linspace(h, 1 - h, N)

    main_diag = 2 + a * h**2
    off_diag = -1 * np.ones(N - 1)
    A = np.diag(main_diag * np.ones(N)) + np.diag(off_diag, -1) + np.diag(off_diag, 1)

    F = np.zeros(N)
    F[-1] += 1

    u = np.linalg.solve(A, F)
    u_ex = solution_exacte(x, a)
    err = np.max(np.abs(u - u_ex))

    # Plot
    plt.plot(x, u, label='Numérique')
    plt.plot(x, u_ex, '--', label='Exacte')
    plt.xlabel('x')
    plt.ylabel('u(x)')
    plt.title(f'Solution pour a = {a}, N = {N}')
    plt.legend()
    plt.grid(True)
    plt.show()

    print(f"Erreur max : {err:.2e}")
    return h, err

## Analyse de l'erreur en fonction de h

In [ ]:
a = 1
N_vals = [10, 20, 40, 80, 160]
hs, erreurs = [], []

for N in N_vals:
    h, err = solve_diffusion_reaction(a, N)
    hs.append(h)
    erreurs.append(err)

# Tracé log-log
plt.loglog(hs, erreurs, 'o-', label='Erreur max')
plt.xlabel('Pas h')
plt.ylabel('Erreur ∞')
plt.title('Erreur vs h (log-log)')
plt.grid(True, which='both', ls='--')
plt.legend()
plt.show()

## Interprétation de l'ordre à partir des rapports d'erreur

In [ ]:
for i in range(len(hs) - 1):
    ratio = erreurs[i] / erreurs[i+1]
    print(f"Erreur(h={hs[i]:.3f}) / Erreur(h={hs[i+1]:.3f}) = {ratio:.2f}")

## Comparaison des normes d'erreur

In [ ]:
# Calcul des erreurs en norme ∞ et 2
for N in N_vals:
    h = 1 / (N + 1)
    x = np.linspace(h, 1 - h, N)
    main_diag = 2 + a * h**2
    off_diag = -1 * np.ones(N - 1)
    A = np.diag(main_diag * np.ones(N)) + np.diag(off_diag, -1) + np.diag(off_diag, 1)
    F = np.zeros(N)
    F[-1] += 1
    u_num = np.linalg.solve(A, F)
    u_exact = solution_exacte(x, a)

    err_inf = np.max(np.abs(u_num - u_exact))
    err_2 = np.linalg.norm(u_num - u_exact, 2)
    print(f"N={N}, h={h:.5f}, ||e||_∞ = {err_inf:.2e}, ||e||_2 = {err_2:.2e}")

## Estimation de la pente log-log (ordre de convergence)

In [ ]:
log_h = np.log(hs)
log_err = np.log(erreurs)
pente = np.polyfit(log_h, log_err, 1)[0]
print(f"Ordre de convergence estimé (pente log-log) : {abs(pente):.2f}")